# ECON408: Midterm Practice Problems

Jesse Perla, UBC

## Student Name/Number: (doubleclick to edit)

### Instructions

-   Edit the above cell to include your name and student number.
-   Submit just this `ipynb` to Canvas. Do not rename, it associates
    your student number with the submission automatically.

In [1]:
using Distributions, Plots, LaTeXStrings, LinearAlgebra, Statistics, Random

## Short Question 1

Given a stochastic process $X_t$, write a paragraph describe what
economists mean by rational expectations? Explain this from the context
of an agent making forecasts and biases in their forecasts.

(double click to edit your answer)

## Short Question 2

Explain what a power-law tail means for a CDF $F(x)$ and counter-cdf
$1-F(x)$. For a stochastic process $X_t$, what features of that
stochastic process might lead to a power-law tail in the stationary
distribution?

(double click to edit your answer)

## Short Question 3

What is a state space model? Explain what type of problem a Kalman
Filter solves. What are the key assumptions on the stochastic processes,
priors, etc. that make it applicable?

(double click to edit your answer)

## Short Question 4

What is the definition of a Martingale? Take the following stochastic
process

$$
X_{t+1} = a + X_t + \epsilon_{t+1}
$$ for some $\epsilon_{t+1}$ which is IID. What values of $a$ and
properties of $\epsilon_{t+1}$ would make this a martingale? If the
variance of $\epsilon_{t+1}$ is $\sigma^2$, then in that case would you
expect there to be a stationary distribution? Why or Why not?

(double click to edit your answer)

## Question 1

Following the notes on [AR(1)
processes](https://julia.quantecon.org/introduction_dynamics/ar1_processes.html)
rather than plotting the distribution as normal instead lets see what
the stationary distribution looks like with simulation.

### Part (a)

From $X_0 = 1.0$ simulate up to $T=2000$ using the process

$$
X_t = a X_{t-1} + b + c W_t
$$

Where $a=0.98, b=0.1, c=0.02$.

In [2]:
T = 2000
X_0 = 1.0
a = 0.98
b = 0.1
c = 0.02
# Add code here

0.02

### Part (b)

On the same graph plot the histogram of those simulated values (i.e.,
$\{X_0, \ldots X_T\}$) then plot the density of the stationary
distribution calculated in closed from in [those
notes](https://julia.quantecon.org/introduction_dynamics/ar1_processes.html#equation-mu-sig-star)
(i.e. create a normal distribution with $\mu^* = b/(1-a)$ and
$v^* = c^2/(1 - a^2)$

Hint: `histogram(X, normed=true)` normalizes the empirical draws so they
are a proper PMF.

In [4]:
# Add code here

### Part (c)

Now, do the same plot using the 10th to 500th observations (i.e.,
$\{X_{10}, \ldots X_{500}\}$)

In [6]:
# Add code here

And then a separate plot using the rest

In [7]:
# Add code here

Compare how these line? Explain why each is better or worse?

(double click to edit your answer)

### Part (d)

Change the `c` parameter to be `0.2`.

Now repeat the display of the two histograms (1) split of the 10th to
500th ; and (2) the remainder

In [10]:
T = 2000
X_0 = 1.0
a = 0.98
b = 0.1
c = 0.2
# Add code here, creating new cells as required

0.2

Compare this to part (d)? Has the results changed and if so, then why?

(double click to edit your answer)

## Question 2

In this problem we will examine wealth dynamics for a simpler setup. Log
income, $\log y_t$ follows an AR(1) process,

$$
\log y_{t+1} = \mu_y + \rho_y \log y_t + \sigma_y \epsilon_{1,t+1}
$$

and

$$
\log R_{t+1} = \mu_r + \sigma_r \epsilon_{2,t+1}
$$

where $\epsilon_{1,t+1} \sim N(0,1)$ and $\epsilon_{2,t+1} \sim N(0,1)$
are IID shocks.

As in the wealth dynamics problem from before, the evolution of wealth
is given by

$$
w_{t+1} = R_{t+1}s(w_t) + y_{t+1}
$$

where $s(w_t)$ is the exogenously given savings function from before,

$$
s(w) =  s_0 w \cdot \mathbb 1\{w \geq \hat w\} = \begin{cases} s_0 w & w \geq \hat{w}\\
0 & w < \hat{w}
\end{cases}
$$

For a constant $s_0 \in (0,1)$ which is savings rate and $\hat{w}\geq 0$
is a minimum wealth threshold below which they do not save and instead
consumer all of their income (since $w_{t+1} = y_{t+1}$ in that case).

As before, take the lorenz and gini code as given,

In [13]:
function lorenz(v)  # assumed sorted vector
    S = cumsum(v)  # cumulative sums: [v[1], v[1] + v[2], ... ]
    F = (1:length(v)) / length(v)
    L = S ./ S[end]
    return (; F, L) # returns named tuple
end
# Assumes that v is sorted!
gini(v) = (2 * sum(i * y for (i,y) in enumerate(v))/sum(v)
           - (length(v) + 1))/length(v)

gini (generic function with 1 method)

Take the following simpler structure for holding the parameters of the
model

In [14]:
function simple_wealth_dynamics_model(;
                 w_hat=1.0, # savings parameter
                 s_0=0.75, # savings parameter
                 mu_y=0.1, # labor income parameter
                 sigma_y=0.1, # labor income parameter
                 rho_y=0.9, # labor income parameter
                 mu_r=0.0, # rate of return parameter
                 sigma_r=0.2, # rate of return parameter
                 )
    return (;w_hat, s_0, mu_y, sigma_y, rho_y, mu_r, sigma_r)
end

simple_wealth_dynamics_model (generic function with 1 method)

and a modified version of the `simulate_panel` function

In [15]:
function simulate_panel(N, T, p; y_0 = p.mu_y/(1-p.rho_y), w_0 = p.mu_y/(1-p.rho_y))
    # Setup initial conditions
    w = w_0 * ones(N) # start at same w_0
    logy = log.(y_0 * ones(N)) # start at same y_0
    logR = zeros(N) # not used in this exact example, but could be generalized
    
    # Preallocate next period states and R intermediates
    w_p = similar(w)
    logR_p = similar(w)
    logy_p = similar(w)    


    # Temporary used in calculations
    savings_proportion = similar(w) # include constant and R_{t+1}

    for t in 1:T
        R_shock = randn(N)
        y_shock = randn(N)
        @inbounds for i in 1:N
            logy_p[i] = p.mu_y + p.rho_y*logy[i] + p.sigma_y*y_shock[i]
            logR_p[i] = p.mu_r + p.sigma_r*R_shock[i] # no autocorrelation but could reference logR[i]
            savings_proportion[i]  = (w[i] >= p.w_hat) ? p.s_0 * exp(logR_p[i]) : 0.0
            w_p[i] = savings_proportion[i]*w[i] + exp(logy_p[i])
        end
        # Step forward
        w .= w_p
        logy .= logy_p
        logR .= logR_p
    end    
    sort!(w) # sorts the wealth so we can calculate gini/lorenz        
    F, L = lorenz(w)
    return (;w, y = exp.(logy_p), F, L, gini = gini(w))
end

simulate_panel (generic function with 1 method)

The following code shows a basic simulation,

In [16]:
p = simple_wealth_dynamics_model()# Or simple_wealth_dynamics_model(;w_hat = 2.0) to swap out a single parameter, etc.
N = 10_000
T = 200
w_0 = 10.0

res = simulate_panel(N, T, p; w_0) # uses the default y_0 but overrides the w_0 default
@show res.gini, mean(res.w), mean(res.y); # show some of the results

(res.gini, mean(res.w), mean(res.y)) = (0.16906036675535943, 11.793935371079193, 2.7780747281098623)

### Part (a)

Using the above code and default parameter values, simulate the model to
see the difference in the gini coefficients where you change the
variance on the returns `sigma_r` 10 points between `0.0` to `0.3`.

Using your simulations, plot the gini coefficient as a function of
`sigma_r` and comment on the results.

In [18]:
# Modify/add code here, creating new cells as required
sigma_r_values = range(0.0, 0.3, 10)
N = 10_000
T = 200

# simple_wealth_dynamics_model(;sigma_r = sigma_r_values[1]) # etc. to create modified models
res = simulate_panel(N, T, simple_wealth_dynamics_model(;sigma_r = sigma_r_values[end])) # for example, this swaps the sigma_r with the last value in the range
@show res.gini;

res.gini = 0.23761959411007538

Can you provide a brief interpretation of the results?

(double click to edit your answer)

### Part (b)

Now, take the same model and lets shut off all variation on the income
process to leave it as a fixed value so $y_{t+1} = y_t$ by setting
`sigma_y = 0, rho_y = 1, mu_y = 0` and then initializing all of the
agents with the `y_0  = 0.5` and `w_0 = 2.0`

Calculate the gini coefficient for the same range of `sigma_r` values as
above and plot the results.

In [20]:
 # Modify/add code here, creating new cells as required
p = simple_wealth_dynamics_model(;sigma_y = 0.0, rho_y = 1.0, mu_y = 0.0)
N = 10_000
T = 1000
y_0 = 5.0
w_0 = 2.0
# res = simulate_panel(N, T, p; y_0, w_0)  # adapt this, passing in the y_0 and w_0

2.0

Can you provide a brief interpretation of the results?

(double click to edit your answer)